<!-- <div style='float:right'><img width=200 src="hse-logo.jpg" alt="HSE logo"></img></div> -->
<div style='float:left'><img width=400 src="python_logo.png" alt="Python"></img></div>

<div style='float:right'>

<h1 align='center'>Язык программирования Python</h1>

<h2 align='right'>Бобер Станислав Алексеевич</h2>
<h3 align='right'>Ст. преп. Департамента Прикладной Математики</h3>
<h3 align='right'>e-mail: sbober@hse.ru, stas.bober@gmail.com</h3>
</div>


<h1 align='center'>Лекция 3. Решение задач при помощи NumPy, SciPy, Matplotlib</h1>

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Пример 0. Найти уравнение параболы по трем точкам

## Пример 1. Найти уравнение окружности по трем точкам

## Пример 2. Посчитать длину ломаной линии

## Пример 3. Посчитать объем под поверхностью, заданной набором значений на регулярной решетке

## Пример 4. Преобразовать рисунок, наложить маску

## Вопросы и дополнения к лекции


[Scientific Python Lectures](https://github.com/jrjohansson/scientific-python-lectures)

[100 Numpy Exercises](https://github.com/rougier/numpy-100/blob/master/100_Numpy_exercises_no_solution.ipynb)